<a href="https://colab.research.google.com/github/galrat/parsing/blob/main/parser_google_espacenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00


In [7]:
from bs4 import BeautifulSoup
from google.colab import files

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd
import requests
import time


# defs

In [8]:
def get_claim_google(soup):
  all = soup.find_all('div', class_='flex flex-width style-scope patent-result')[-1].find_all('div', class_='claim-text style-scope patent-text')[1]
  return all.text

def get_claim_google_2(soup):
  try:
      # all
      try:
          try:
              all = soup.find('div', class_='claim-text style-scope patent-text').find('span', class_='notranslate style-scope patent-text').text
          except:
              all = soup.find('div', class_='claim-text style-scope patent-text').text
          #print('all', all)
      except:
          all = soup.find('section', id='claims').find('span', class_='notranslate style-scope patent-text').text
          #print('all', all)

      # original
      try:
          original = soup.find('div', class_='claim-text style-scope patent-text').find('span',
                                                                                        class_='notranslate style-scope patent-text').find(
              'span').text
      except:
          try:
              original = soup.find('section', id='claims').find('span',
                                                                class_='notranslate style-scope patent-text').find(
                  'span').text
          except:
              original = ''
      #print('original', original)

      # translation
      claim_1 = all.replace(original, '')
      #print('claim 1:', claim_1)
  except:
      claim_1 = 'no_data'
  if claim_1 == '' or claim_1 == 'no_data':
    try:
      claim_1 = soup.find_all('div', class_='flex flex-width style-scope patent-result')[-1].find_all('div', class_='claim-text style-scope patent-text')[1].text
    except:
      claim_1 = 'no_data'
  return claim_1

In [9]:
def get_data_from_google_patent(soup):

  # title
  title = soup.find('div', id='wrapper').find('h1', id='title').text.replace('\n', '').strip()
  #print('title:', title)

  # patent_number
  patent_number = soup.find('h2', id='pubnum').text.strip().replace('\n', '').replace('\t', '')
  #print('patent_number:', patent_number)

  # applicant
  author_counter = 0
  applicant_counter = 0
  counter = 0
  author_check = 0
  applicant_check = 0
  important_people_data = soup.find('dl', class_='important-people style-scope patent-result')
  for i in important_people_data.find_all('dt', class_='style-scope patent-result'):
      if 'Inventor' in i.text:
          author_counter = counter
          author_check = 1
      if 'Assignee' in i.text:
          applicant_counter = counter
          applicant_check = 1
      counter += 1
  #print('author_counter', author_counter, 'applicant_counter', applicant_counter)

  authors = ''
  if author_check == 1:
      try:
          authors_data = soup.find('dl', class_='important-people style-scope patent-result')
          authors_data = authors_data.find_all('dt', class_='style-scope patent-result')[
              author_counter].find_next_siblings('dd')
          for author in authors_data:
              if '\n' in author.text:
                  break
              authors = authors + author.text + ';'
          authors = authors.replace('\n', '')
      except:
          #print('no authors data')
          authors = ('no authors data')
  else:
      authors = 'no_data'

  applicants = ''
  if applicant_check == 1:
      try:
          applicants_data = soup.find('dl', class_='important-people style-scope patent-result')
          applicants_data = applicants_data.find_all('dt', class_='style-scope patent-result')[
              applicant_counter].find_next_siblings('dd')
          for applicant in applicants_data:
              applicants = applicants + applicant.text.strip() + ';'
          applicants = applicants.replace('\n', '').strip()
      except:
          #print('no applicants data')
          applicants = ('no applicants data')
  else:
      applicants = 'no_data'

  # filing_date
  filind_date = 'no_data'
  publication_date = 'no_data'
  status = 'no_data'
  dates_data = soup.find_all('div', class_='event layout horizontal style-scope application-timeline')
  for data in dates_data:
      if 'filed by' in data.text:
          filind_date = data.text.split('Application')[0]
          #print('filing date:', filind_date)
      if 'Publication' in data.text:
          publication_date = data.text.split('Publication')[0]
          #print('Publication date:', publication_date)
      if 'Status' in  data.text:
          status = data.text.split('Status')[1].replace('\n', '').replace('\t', '')
          #print('Status:', status)

  # application number
  try:
      application_number = soup.find_all('div', class_='header style-scope application-timeline')[1].text
      application_number = application_number.split('Application')[1].split('events')[0].replace(' ','')
  except:
      print('no application number data')
      application_number = 'no data'
  #print('application_number', application_number)


  patent_type = 'no_data'
  #print('ipc text', soup.find('div', class_='style-scope classification-viewer').find('div', 'style-scope classification-tree').text)

  ipc_data = 'no_data'
  try:
      ipcs = soup.find('div', class_='style-scope classification-viewer').find_all('div', class_='style-scope classification-tree')
      for ipc in ipcs:
          ipc_class = ipc.find(first=True).text.strip().replace('\n', '').replace('\t', '')
          #print('ipc', ipc_class)
      #print(ipc_class)
  except:
      ipc_data = 'no_data'

  ipc_datas = soup.find_all('div', class_='style-scope classification-tree')
  for i in ipc_datas[:1]:
      target = i.find_all(class_='code style-scope classification-tree')
      #print('target ipc', target[-1].text)
      # print(i.text.strip())
      ipc_data = target[-1].text

  # claim 1
  claim_1 = ''
  try:
    claim_1 = get_claim_google_2(soup)
    #print('claim_1 google', claim_1)
  except:
    print('no claims')

  # abstract
  abstract = ''
  try:
    abstract_orig = soup.find('section', id='abstract').find('span', class_='google-src-text style-scope patent-text').text#.text.replace('\n', '').replace('\t', '').replace('Abstracttranslated from ', '')
    #print('abstract_orig', abstract_orig)
    abstract_all = soup.find('section', id='abstract').text
    #print('abstract_all', abstract_all)
    abstract_eng = abstract_all.replace(abstract_orig, '').strip()
    abstract_eng = abstract_eng.replace('\n', '')
    #print('abstract_eng', abstract_eng)
    abstract = abstract_eng
  except:
    abstract = soup.find('section', id='abstract').text.replace('\n', '')


  # other applications county code
  others = ''
  other_app = soup.find('div', class_='wrap style-scope application-timeline').find('div',
                                                                                    class_='event style-scope application-timeline').find_all(
      'a')
  for other in other_app:
      country_code = other.text.strip()
      others += country_code + ','
  #print(others)

  printing = 0
  while printing:
      print('patent_number:', patent_number)
      print('application_number:', application_number)
      print('title:', title)
      print('type:', patent_type)
      print('filind_date:', filind_date)
      print('applicant:', applicants)
      print('authors:', authors)
      print('ipc_data:', ipc_data)
      print('publication_date:', publication_date)
      print('abstract:', abstract)
      print('claim_1:', claim_1)
      print('status:', status)
      print('others:', others)
      break

  header = ['patent number', 'application_number', 'title', 'INV/UM', 'filing_date', 'applicant', 'autors',
                          'publiction_date', 'ipc', 'claims', 'status', 'abstract', 'other_applications']
  data_line = []
  data_line.append(patent_number)
  data_line.append(application_number)
  data_line.append(title)
  data_line.append(patent_type)
  data_line.append(filind_date)
  data_line.append(applicants)
  data_line.append(authors)
  data_line.append(publication_date)
  data_line.append(ipc_data)
  data_line.append(claim_1.replace('\n', ''))
  data_line.append(status)
  data_line.append(abstract.replace('\n', ''))
  data_line.append(others[:-1])
  return data_line


In [10]:
def get_from_espacenet(espacenet_url):
    #print('def', espacenet_url)
    data_line2 = []

    options = Options()
    options.add_argument("--headless")
    options.add_argument('--no-sandbox')
    options.add_argument('--user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36')
    driver = webdriver.Chrome(options=options)
    driver.get(espacenet_url)
    time.sleep(2)
    WebDriverWait(driver, 20).until(ec.visibility_of_element_located((By.ID, 'pagebody')))

    soup = BeautifulSoup(driver.page_source, 'lxml')

    #print(soup)
    #print('---------------------\n')
    # title
    title = soup.find('h3').text.strip().replace('\n', '').replace('\t', '')
    #print('title:', title)
    table = soup.find('table', class_='tableType3')
    #print(table.text.strip())
    app_num = soup.find('h1', class_='noBottomMargin').text.split('―')[0].split('\n')[-1].strip().replace('\xa0', '').replace('(', '').replace(')', '')

    applicant = ''
    authors = ''
    ipc = ''
    priority = ''
    original_application = ''
    others= ''

    table_lines = table.find_all('tr')
    for line in table_lines:
      if 'Applicant(s):' in line.text:
        applicant = ' '.join(line.find('td').text.replace('\n', '').replace('\t', '').strip().split())
        applicant = applicant.split('+')[0].strip()
        #print('applicant:', applicant)

      if 'Inventor(s):' in line.text:
        authors = ' '.join(line.find('td').text.replace('\n', '').replace('\t', '').strip().split())
        authors = authors.split('+')[0].strip()
        #print('authors:', authors)

      if 'Classification:' in line.text:
        ipc = ' '.join(line.find('td').text.replace('\n', '').replace('\t', '').strip().split()).split('- cooperative')[0]
        ipc = ipc.replace('- international:', '').strip()
        ipc = ipc.split('(IPC1-7)')[0].strip()
        #print('ipc:', ipc)

      if 'Priority number(s):' in line.text:
        priority = ' '.join(line.find('td').text.replace('\n', '').replace('\t', '').strip().split()).split('- cooperative')[0]
        #print('Priority:', priority)

      if 'Application number:' in line.text:
        original_application = ' '.join(line.find('td').text.replace('\n', '').replace('\t', '').strip().split()).split('- cooperative')[0]
        original_application = original_application.replace('Global Dossier', '').strip()
        #print('original_application:', original_application)

      if 'Also published as:' in line.text:
        others = ' '.join(line.find('div', id='lessPublishedAs').text.replace('\n', '').replace('\t', '').strip().split()).split('- cooperative')[0]
        #others = soup.find('div', id='lessPublishedAs').text.strip()
        #print('others:', others)


    publication = 'no_data'

    abstract = soup.find('div', class_='application article clearfix').find('p').text
    #print(abstract)

    printing = 0
    while printing:
        print('title:', title)
        print('applicant:', applicant)
        print('authors:', authors)
        print('ipc:', ipc)
        print('priority:', priority)
        print('original_application:', original_application)
        print('publication:', publication)
        print('abstract:', abstract)
        print('others:', others)
        break

    data_line2.append(app_num)
    data_line2.append(applicant)
    data_line2.append(authors)
    data_line2.append(ipc)
    data_line2.append(priority)
    data_line2.append(original_application)
    data_line2.append(publication)
    data_line2.append(title)
    data_line2.append(abstract)
    data_line2.append(others)

    header = ['patent_number2', 'applicant2', 'authors2', 'ipc2', 'priority2', 'original2', 'publication2', 'title', 'abstract',
              'other_applications']

    return (data_line2)

# code

# ниже надо ввести номера патентов для парсинга

In [11]:
# original numbers for parsing
numbers = '''CN-107856831-B
CN-104611491-A
CN-108559395-A
'''

In [12]:
numbers = numbers.replace('-', '')
all_numbers = numbers.split()
print('lenght of all_numbers', len(all_numbers))

lenght of all_numbers 3


In [13]:
file_for_saving_results = "/content/position of charging machine.txt"

header_1 = ['patent number', 'applicant2', 'authors2', 'ipc2', 'priority2', 'original2', 'publication2', 'title_2', 'abstract_2',
              'other_applications_2']
header_2 = ['patent number', 'application_number', 'title', 'INV/UM', 'filing_date', 'applicant', 'authors',
                            'publiction_date', 'ipc', 'claims', 'status', 'abstract', 'other_applications']
try:
    saved_data = pd.read_csv(file_for_saving_results, sep='\t')
    parsed_numbers = saved_data['original_number'].values
except:
    parsed_numbers = []
    with open(file_for_saving_results, 'a', encoding="utf-8") as file:
        file.writelines('\t'.join(['original_number'] + header_2 + header_1) + '\n')
    print("creating new file")

remaining_numbers = list(set(all_numbers) - set(parsed_numbers))
print('remaining_numbers', remaining_numbers)
print('total remaining', len(remaining_numbers))

creating new file
remaining_numbers ['CN108559395A', 'CN104611491A', 'CN107856831B']
total remaining 3


In [14]:
# correct app_numbers
remaining_numbers_corrected = []
for number in remaining_numbers:
    if number[:2] == 'WO' and len(number) == 12:
        number = 'WO20' + number[2:] + '/en'
    remaining_numbers_corrected.append(number)

final_urls = []
for app_num in remaining_numbers_corrected:
  if app_num[:3] == 'US2' and len(app_num) == 14:
      url =  app_num[:].replace('-', '')[:6] + '0' + app_num[:].replace('-', '')[6:] + '/en'
      url = 'https://patents.google.com/patent/' + url
      #print('corrected url', url)
  else:
      url = 'https://patents.google.com/patent/' + app_num[:].replace('-', '').replace('\n', '') + '/en'
      #print('url', url)
  final_urls.append(url)

print(final_urls)


['https://patents.google.com/patent/CN108559395A/en', 'https://patents.google.com/patent/CN104611491A/en', 'https://patents.google.com/patent/CN107856831B/en']


In [15]:
# get data by url
counter = 1
for original_number, url in zip(remaining_numbers, final_urls[:]):
  print(counter, '/', len(remaining_numbers))
  counter += 1
  print(url)
  try:
    options = Options()
    options.add_argument("--headless")
    options.add_argument('--no-sandbox')
    options.add_argument('--user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36')
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    WebDriverWait(driver, 20).until(ec.visibility_of_element_located((By.ID, 'content')))
    soup = BeautifulSoup(driver.page_source, 'lxml')
    data_line_google = get_data_from_google_patent(soup)
    print('data_line_google', data_line_google)

    # get espacenet url
    all_additional_urls = [url.get('href') for url in soup.find('dl', class_='links style-scope patent-result').findAll('a')]
    for url in all_additional_urls:
      if 'espacenet' in url:
        #print('espacenet_url from google', url)
        espacenet_url = url
        break

    # get data using espacent url
    data_line_espacenet = get_from_espacenet(espacenet_url)
    print('data_line_espacenet', data_line_espacenet)
    print()

    with open(file_for_saving_results, 'a', encoding="utf-8") as file:
        file.writelines('\t'.join([original_number] + data_line_google + data_line_espacenet) + '\n')
        #print('writing')
  except:
    print('smth wrong with------------------------------------->', original_number)
  #break
print('====================finish=========================')

1 / 3
https://patents.google.com/patent/CN108559395A/en
data_line_google ['CN108559395A', 'CN201810444863.6A', 'It is a kind of to keep a public place clean fresh-keeping anti-fouling agent and surface treatment method for ceramic tile', 'no_data', '2018-05-10', 'Foshan Sanshui Planck New Material Co Ltd;', '姚燕春;', '2021-08-24', 'C09D183/04', '1. a kind of keep a public place clean fresh-keeping anti-fouling agent for ceramic tile, it is characterised in that：Including the raw material counted in parts by weight as follows：20~98 parts of alkoxy-modified silicone oil, 0~40 part of hydroxy silicon oil, 0~40 part of silane coupling agent, 0~15 part of silica, 0.5~20Part organotin catalysts.', 'Active', 'Abstracttranslated from Keep a public place clean fresh-keeping anti-fouling agent and surface treatment method for ceramic tile the invention discloses a kind of.The anti-fouling agent includes the following raw material counted in parts by weight：20~98 parts of alkoxy-modified silicone o

In [16]:
# download the created file
files.download(file_for_saving_results)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>